In [3]:
import tools.architectures as nn
import tools.processing as pre
import tensorflow as tf

text = pre.get_text("data/prepped/clean2_pac.txt")[:30000]
vocab = pre.Vocabulary(text)

class MultiLayerEmbeddedLabels(nn.Trainable):
    def __init__(self, name):
        super().__init__(name)

    def build(self, num_layers, hidden_layer_size, vocab_size, time_steps, embedding):
        self.time_steps = time_steps
        self.vocab_size = vocab_size

        self.Y = tf.placeholder(tf.int32, shape=[None, vocab_size], name="labels")

        self.X = tf.placeholder(tf.int32, shape=[None, time_steps], name="data")
        self.embed, self.embeddings = embedding.output(self.X)


        with tf.variable_scope(self.name, reuse=tf.AUTO_REUSE):

            self.stacked_cells = nn.lstm_layer(num_layers, hidden_layer_size)

            self.outputs, self.states = tf.nn.dynamic_rnn(
                self.stacked_cells, self.embed, dtype=tf.float32
            )

            self.last_rnn_output = self.states[num_layers - 1][1]

            self.final_output, W_out, b_out = nn.full_layer(self.last_rnn_output, embedding.embedding_dimension)

            self.weights.append(W_out)
            self.biases.append(b_out)
            
            self.embed_labels  = tf.nn.embedding_lookup( self.embeddings, self.Y )
            # self.embed_preds   = tf.nn.embedding_lookup( self.embeddings, self.final_output )
            
            self.loss = tf.reduce_mean(tf.square(self.final_output - self.embed_labels))

            self.optimizer = tf.train.AdamOptimizer()
            self.train_step= self.optimizer.minimize(self.loss)

            self.correct_prediction = tf.equal(tf.argmax(self.Y,1), tf.argmax(self.final_output, 1))
            self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))*100

In [4]:
text = pre.get_text("data/prepped/clean2_pac.txt")[:60000]
vocab = pre.Vocabulary(text)

NUM_LAYERS = 1
HIDDEN_LAYER_SIZE = 256
VOCAB_SIZE = vocab.get_size()
TIMESTEPS = 20

EPOCHS = 20
BATCH_SIZE = 256

EMBEDDING_SIZE = 128


data, labels = vocab.making_embedded_one_hot(text, TIMESTEPS)

embedding = nn.LeanableEmbedding(name = "learnable-embedding-2")
embedding.build(VOCAB_SIZE, EMBEDDING_SIZE)

rnn = MultiLayerEmbeddedLabels(name = "multi-pac-total")
rnn.build(NUM_LAYERS, HIDDEN_LAYER_SIZE, VOCAB_SIZE, TIMESTEPS, embedding)

nn.train(rnn, data, labels, vocab, epochs=EPOCHS, batch_size=BATCH_SIZE, temperature=.6, embedding=True)

ResourceExhaustedError: OOM when allocating tensor with shape[13921,2099,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node multi-pac-total_1/embedding_lookup (defined at <ipython-input-3-65901a78435d>:37)  = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@embedding_lookup_1"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable_1/read, _arg_labels_1_0_1, multi-pac-total_1/embedding_lookup/axis)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'multi-pac-total_1/embedding_lookup', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/frankzl/.envs/env36-ml/.venv/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <modu

In [4]:
labels.shape

(13921, 2099)

In [5]:
data.shape

(13921, 20)

In [3]:

_data, _labels = vocab.making_full_one_hot(text, TIMESTEPS)


In [5]:
_labels

array([[  18.],
       [  19.],
       [  20.],
       ...,
       [ 201.],
       [   4.],
       [2097.]])